In [1]:
import json
import pickle
import numpy as np
import torch
import clip

model, preprocess = clip.load('ViT-L/14', device='cuda')

/home/scm22/miniconda3/envs/normal/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
threshold = 10

with open('concept_bank/object_synsets.json', 'r') as file:
    objects = json.load(file)
with open('concept_bank/attribute_synsets.json', 'r') as file:
    attributes = json.load(file)
with open('concept_bank/synsets.json', 'r') as file:
    synsets = json.load(file)

vg_concepts = {}
for c in objects.values():
    if (c.split(".")[1] == 'n' or c.split(".")[1] == 'a') and len(c.split(".")[0]) > 2:
        cpt = c.split(".")[0].replace("_", " ")
        if cpt in vg_concepts:
            vg_concepts[cpt] += 1
        else:
            vg_concepts[cpt] = 1

for c in attributes.values():
    if (c.split(".")[1] == 'n' or c.split(".")[1] == 'a') and len(c.split(".")[0]) > 2:
        cpt = c.split(".")[0].replace("_", " ")
        if cpt in vg_concepts:
            vg_concepts[cpt] += 1
        else:
            vg_concepts[cpt] = 1

for c in synsets:
    if (c['synset_name'].split(".")[1] == 'n' or c['synset_name'].split(".")[1] == 'a') and len(c['synset_name'].split(".")[0]) > 2:
        cpt = c['synset_name'].split(".")[0].replace("_", " ")
        if cpt in vg_concepts:
            vg_concepts[cpt] += 1
        else:
            vg_concepts[cpt] = 1

concepts = []
for cpt, times in vg_concepts.items():
    if times >= threshold:
        concepts.append(cpt)
concepts = sorted(concepts)#[3:]

In [15]:
concept_bank = {}

model.eval()
with torch.no_grad():
    for cpt in concepts:
        text = clip.tokenize(cpt).cuda()
        text_features = model.encode_text(text).cpu()
        text_features = text_features / text_features.norm()
        concept_bank[cpt] = text_features[0]

with open(f'concept_bank/concept_bank_num_{len(concept_bank)}.pkl', 'wb') as file:
    pickle.dump(concept_bank, file)